# TMP2


<span style="color: red;">no context in args of Ghostbuster save/update/restore_agent, agent reation;<\span>
    
<span style="color: red;">defined at -2 for everywhere<\span>

## [Repast for Python (Repast4Py) User Guide](https://repast.github.io/repast4py.site/guide/user_guide.html)

## [API](https://repast.github.io/repast4py.site/apidoc/index.html)

## [GitHub Repast/repast4py](https://github.com/Repast/repast4py)

## [MPI for Python](https://mpi4py.readthedocs.io/en/stable/tutorial.html#collective-communication)


# An idea for the initial example: 

### something close to the [Chakraborti model](https://www.scientificamerican.com/article/is-inequality-inevitable/) of Winners, Losers 

ESC $\ell$ set or unset row numbers

the script in next cell avoids scrolling outputs

In [1]:
%%javascript
// to avoid scroll in windows
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## -2

MPI and context for everywhere

In [2]:
%%writefile MPIandContext.py

import time
from mpi4py import MPI
from repast4py import context as ctx
import repast4py 
from repast4py import parameters

comm = MPI.COMM_WORLD
rank    = comm.Get_rank()
rankNum = comm.Get_size() #pt

# create the context to hold the agents and manage cross process
# synchronization
context = ctx.SharedContext(comm)

# https://repast.github.io/repast4py.site/apidoc/source/repast4py.parameters.html
"""
create_args_parser()
Creates an argparse parser with two arguments: 
1) a yaml format file containing model parameter input, and 
2) an optional json dictionary string that can override that input.
"""
parser = parameters.create_args_parser()

args = parser.parse_args()
#print(1,args,flush=True)

"""
init_params(parameters_file, parameters)
Initializes the repast4py.parameters.params dictionary with the model input parameters.
"""
params = parameters.init_params(args.parameters_file, args.parameters)
#print(2,params,flush=True)

repast4py.random.init(rng_seed=params['myRandom.seed'][rank])
rng = repast4py.random.default_rng 

#timer T()
startTime=-1
def T():
    global startTime
    if startTime < 0:
        startTime=time.time()
    return time.time() - startTime



    

Overwriting MPIandContext.py


## -1

memory allocations to manage agents

In [3]:
%%writefile memAlloc.py

agent_cache={} # dict with uid as keys and agents' tuples as values, 
               # used by restore_agent (def in classes.py) to avoid rebuild agents
               # IS IT IN OUR CASE USEFUL?
    
ghostsToRequest=[] # list of tuples containing for each ghost the uid and its current rank;
                   # used by search function of
                   # GhostBuster class
                   # and by requestGhosts(self) function of the model


Overwriting memAlloc.py


## 0

program starter

In [4]:
%%writefile starter.py

from runMod import *



run(params)

Overwriting starter.py


## 1 

run the model

In [5]:
%%writefile runMod.py

from typing import Dict
from model import *

def run(params: Dict):
    #print(3, params)
    #print (4, params['random.seed'])
    #print (4.1, params['a'])
    
    model = Model(params) #Model(MPI.COMM_WORLD, params)
    model.start()

Overwriting runMod.py


## 2

classes (agent definitions + restore_agent function)

In [6]:
%%writefile classes.py

from repast4py import core
from typing import Tuple, List
import json
#from repast4py import context as ctx
from memAlloc import *
from MPIandContext import *


class WinnerLoser(core.Agent):

    TYPE = 0
    
    def __init__(self, local_id: int, rank: int, wallet: float):
        super().__init__(id=local_id, type=WinnerLoser.TYPE, rank=rank)

        self.myWallet = wallet
        self.counterpartRank = -1
        self.counterpartLocalId = -1
        
        self.haveGhost = [False] * rankNum
        self.haveGhost[rank] = True
        
    def requestingGhostIfAny(self) -> List:
        self.counterpartRank = int(rng.integers(0,rankNum))
        if not self.haveGhost[self.counterpartRank]:
            self.haveGhost[self.counterpartRank] = True
            return [self.counterpartRank, ((self.uid[0], self.TYPE, rank), rank)]
        
        
        
        
    def lookForMinWallet(self,agSet):

        agWalletSet=list(aWinnerLoser.myWallet for aWinnerLoser in agSet)
        self.minWalletPosition=agWalletSet.index(min(agWalletSet))
        
    def give(self,agSet):
        
        list(agSet)[self.minWalletPosition].myWallet+=1
        self.myWallet-=1
        
    def save(self) -> Tuple: # mandatory
        """
        Saves the state of the WinnerLoser as a Tuple.

        Returns:
            The saved state of this WinnerLoser.
        """
        print("quiiiiiiiiiiiiiiiiiiiiiiii myWallet", self.myWallet, flush=True)
        return (self.uid, self.myWallet)

    def update(self, wallet: float): # mandatory
        """
        Updates the state of this agent when it is a ghost
        agent on some rank other than its local one.
        """
        self.myWallet=wallet
        print("quaaaaaaaaaaaaaaaaaaaaaaaa", flush=True)

        
      
            
def restore_agent(agent_data: Tuple):

    uid=agent_data[0]

    if uid[1] == WinnerLoser.TYPE:
        print("quoooooooooooooooooooooooooooo", flush=True)
    
        if uid in agent_cache:   # look for agent_cache in model.py
            tmp = agent_cache[uid] # found
            tmp.myWallet = agent_data[1] #restore data

        else: #creation of an instance of the class with its data
            tmp = WinnerLoser(uid[0], uid[2],agent_data[1])                
            agent_cache[uid] = tmp

        return tmp

    

Overwriting classes.py


## 3

the model

In [7]:
%%writefile model.py

#from mpi4py import MPI
from MPIandContext import *
from typing import Dict
from repast4py import schedule
#from repast4py import context as ctx
import repast4py
import json
import numpy as np
from classes import *
from memAlloc import *

def countDigits(n):
    count = 0
    while(n>0):
        count+=1
        n=n//10
    return count


class Model:
    """
    The Model class encapsulates the simulation, and is
    responsible for initialization (scheduling events, creating agents,
    and the grid the agents inhabit IF ANY), and the overall iterating
    behavior of the model.

    Args:
        comm: the mpi communicator over which the model is distributed.
        params: the simulation input parameters
    """
    def __init__(self, params: Dict):

        #self.rank    = comm.Get_rank()
        #self.rankNum = comm.Get_size() #pt
        
        #self.comm = comm
        
        #print(5, "rank", self.rank, "rank number", self.rankNum)
        
        self.mToBcast = []
        
        # create the context to hold the agents and manage cross process
        # synchronization
        #self.context = ctx.SharedContext(comm)
        
        # create the schedule
        # https://repast.github.io/repast4py.site/apidoc/source/repast4py.schedule.html
                
        """
        init_schedule_runner(comm)
        Initializes the default schedule runner, a dynamic schedule of executable 
        events shared and synchronized across processes.
        Events are added to the scheduled for execution at a particular tick. 
        The first valid tick is 0. Events will be executed in tick order, earliest 
        before latest. Events scheduled for the same tick will be executed in the 
        order in which they were added. If during the execution of a tick, 
        an event is scheduled before the executing tick (i.e., scheduled to occur in 
        the past) then that event is ignored. The scheduled is synchronized across 
        process ranks by determining the global cross-process minimum next scheduled 
        event time, and executing only the events schedule for that time. In this way, 
        no schedule runs ahead of any other.
        """
        self.runner = schedule.init_schedule_runner(comm)
        
        """
        schedule_repeating_event(at, interval, evt)
        Schedules the specified event to execute at the specified tick, and repeat at 
        the specified interval.

        Parameters
        at (float) – the time of the event.
        interval (float) – the interval at which to repeat event execution.
        evt (Callable) – the Callable to execute when the event occurs.

            A callable is anything that can be called.
            The built-in callable (PyCallable_Check in objects.c) checks if the argument 
            is either:
                an instance of a class with a __call__ method or
                is of a type that has a non null tp_call (c struct) member which 
                indicates callability otherwise (such as in functions, methods etc.)
        """
        self.runner.schedule_repeating_event(0, 1, self.agentsChoosingCounterpart)
        self.runner.schedule_repeating_event(0.3, 1, self.sync)
        
        """
        schedule_stop(at)
        Schedules the execution of this schedule to stop at the specified tick.

        Parameters
        at (float) – the tick at which the schedule will stop.
        """
        self.runner.schedule_stop(params['stop.at'])
        
        self.runner.schedule_end_event(self.finish)
        

        
        # create agents
        # winnerLoser agents
        
        for i in range(params['WinnerLoser.count'] // rankNum): 
                                                #to subdivide the total #pt
            # create and add the agent to the context
            aWallet=10 * rng.random()
            #print(aWallet,flush=True)
            aWinnerLoser = WinnerLoser(i,rank,aWallet)
            context.add(aWinnerLoser)
            

        
    def agentsChoosingCounterpart(self):        
        
        """
        agents(agent_type=None, count=None, shuffle=False)
        Gets the agents in this SharedContext, optionally of the specified type, count 
        or shuffled.

        Parameters
        agent_type (int) – the type id of the agent, defaults to None.
        count (int) – the number of agents to return, defaults to None, meaning return 
        all the agents.shuffle (bool) – whether or not the iteration order is shuffled.
        If true, the order is shuffled. If false, the iteration order is the order of 
        insertion.

        Returns
        An iterable over all the agents in the context. If the agent_type is 
        not None then an iterable over agents of that type will be returned.

        Return type
        iterable 
        pt addendum: it is a generator, not a list
        """

        tick = self.runner.schedule.tick       
        """
        print("tick",tick,"rank",rank,"clock",T(),\
                            "\nwallets",list(aWinnerLoser.myWallet\
                            for aWinnerLoser in context.agents(agent_type=0)),\
                            flush=True)
        """
        
        del self.mToBcast 
        self.mToBcast = [rank] 
        
        for aWinnerLoser in context.agents(agent_type=0):
            aRequest = aWinnerLoser.requestingGhostIfAny()
            if aRequest != None: self.mToBcast.append(aRequest)
    
        print(self.mToBcast, flush = True)
        self.requestGhosts()
           
        
    def requestGhosts(self): 
        tick = self.runner.schedule.tick
        
        print("@@@@@@@@ tick",tick,"rank", rank)
        """
        #building the data matrix to be broadcasted
        if rank==0:        # example [0,[0,((0,0,1),1)],[1,((0,1,0),0)],[2]] with rankNum => 3
            self.mToBcast=[0,[0,((0,0,1),1)]]
            for k in range(2,rankNum):
                self.mToBcast.append([k])
    

        if rank!=0:        # example [1|2,[0],[1],[2]] with rankNum -> 3
            self.mToBcast=[rank]
            for k in range(rankNum):
                self.mToBcast.append([k])
        """
       
        
        n=params['WinnerLoser.count'] // rankNum
        countB = 10+n*(22+countDigits(n))
        str_countB="S"+str(countB)
        
        print(self.mToBcast, flush = True)
    
        self.mToBcast=json.dumps(self.mToBcast)
        self.mToBcast=np.array(self.mToBcast, dtype=str_countB) 
        self.mToBcast=self.mToBcast.tobytes()    
        
        data=[""]*rankNum
        for k in range(rankNum):
            if rank == k:
                data[k] =self.mToBcast
            else:
                data[k] = bytearray(countB) 
        for k in range(rankNum):
            comm.Bcast(data[k], root=k)

        for k in range(rankNum):
            data[k]=data[k].decode().rstrip('\x00')

        for k in range(rankNum):
            data[k]=json.loads(data[k])
            

        for anItem in data:
            anItem.pop(0)
            for aSubitem in anItem: 
                if len(aSubitem)>1 and aSubitem[0]==rank: 
                    aaSubitem = aSubitem[1][0]
                    aaSubitem = tuple(aaSubitem)
                    aSubitem=(aSubitem[0], (aaSubitem, aSubitem[1][1]))
                    
                    if not tuple(aSubitem[1]) in ghostsToRequest:
                        ghostsToRequest.append(tuple(aSubitem[1]))
        print(rank, "!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", ghostsToRequest,\
             flush=True)
        
        
        """
        https://repast.github.io/repast4py.site/apidoc/source/repast4py.context.html
        request_agents(requested_agents, create_agent)
        Requests agents from other ranks to be copied to this rank as ghosts.

        !!!! This is a collective operation and all ranks must call it, regardless 
        of whether agents are being requested by that rank. The requested agents 
        will be automatically added as ghosts to this rank.

        Parameters
        requested_agents (List) – A list of tuples specifying requested 
        agents and the rank to request from. Each tuple must contain the agents 
        unique id tuple and the rank, for example ((id, type, rank), requested_rank).

        create_agent (Callable) – a Callable that can take the result of an agent 
        save() and return an agent.

        Returns
        ***The list of requested agents.

        Return type
        List[_core.Agent]
        """
        context.request_agents(ghostsToRequest,restore_agent)
        print("***tick",tick,"rank", rank, "agent_cache", agent_cache, flush=True)
        print("***tick",tick,"rank", rank, "ghostsToRequest", ghostsToRequest, flush=True)
        

        
    def sync(self):
        """
        synchronize(restore_agent, sync_ghosts=True)
        Synchronizes the model state across processes by moving agents, 
        filling projection buffers with ghosts, updating ghosted state and so forth.

        Parameters
        restore_agent (Callable) – a callable that takes agent state data and returns 
        an agent instance from that data. The data is a tuple whose first element 
        is the agent’s unique id tuple, and the second element is the agent’s state, 
        as returned by that agent’s type’s save() method.

        sync_ghosts (bool) – if True, the ghosts in any SharedProjections and 
        value layers associated with this SharedContext are also synchronized. 
        Defaults to True.
        """
        context.synchronize(restore_agent)
    
                        
    def finish(self):
        tick = self.runner.schedule.tick
        print("ciao by rank",rank,"at tick",tick,"clock",T(),flush=True)
        
    def start(self):
        self.runner.execute()
        

Overwriting model.py


# Execution

In [8]:
# R U N
# %sx mpirun -n 1 python3 starter.py winners-losers.yaml
! mpirun -n 3 python3 starter.py winners-losers.yaml
#! /usr/bin/mpirun -n 3 python3 starter.py winners-losers.yaml
# %sx mpirun -n 1 python3 starter.py winners-losers.yaml '{"a": 123,"random.seed": 1000}'

[0, [2, ((0, 0, 0), 0)], [2, ((1, 0, 0), 0)], [1, ((2, 0, 0), 0)]]
@@@@@@@@ tick 0 rank 0
[0, [2, ((0, 0, 0), 0)], [2, ((1, 0, 0), 0)], [1, ((2, 0, 0), 0)]]
[1, [0, ((0, 0, 1), 1)], [2, ((1, 0, 1), 1)], [0, ((2, 0, 1), 1)], [0, ((3, 0, 1), 1)]]
[2, [1, ((0, 0, 2), 2)], [0, ((1, 0, 2), 2)], [0, ((2, 0, 2), 2)], [1, ((3, 0, 2), 2)]]
@@@@@@@@ tick 0 rank 2
[2, [1, ((0, 0, 2), 2)], [0, ((1, 0, 2), 2)], [0, ((2, 0, 2), 2)], [1, ((3, 0, 2), 2)]]
@@@@@@@@ tick 0 rank 1
[1, [0, ((0, 0, 1), 1)], [2, ((1, 0, 1), 1)], [0, ((2, 0, 1), 1)], [0, ((3, 0, 1), 1)]]
0 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [((0, 0, 1), 1), ((2, 0, 1), 1), ((3, 0, 1), 1), ((1, 0, 2), 2), ((2, 0, 2), 2)]
2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [((0, 0, 0), 0), ((1, 0, 0), 0), ((1, 0, 1), 1)]
1 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! [((2, 0, 0), 0), ((0, 0, 2), 2), ((3, 0, 2), 2)]
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 8.856282620266805
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 9.371277593532279
quiiiiiiiiiiii

quiiiiiiiiiiiiiiiiiiiiiiii myWallet 8.856282620266805
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 8.856282620266805
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 2.787845356075669
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 2.2682186762780256
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 2.2682186762780256
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 9.028550747283012
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 9.423522251504036
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 3.667046569571375
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 9.371277593532279
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 0.33516198642458805
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 9.371277593532279
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 0.33516198642458805
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 5.915507813433745
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 5.915507813433745
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 4.7118973513793785
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 8.844623429064837
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 8.844623429064837
quiiiiiiiiiiiiiiiiiiiiiiii myWallet 0.6981076426667698
quaaaaaaaaaaaaaaaaaa

***tick 5 rank 0 agent_cache {(0, 0, 1): Agent(0, 0, 1), (2, 0, 1): Agent(2, 0, 1), (3, 0, 1): Agent(3, 0, 1), (1, 0, 2): Agent(1, 0, 2), (2, 0, 2): Agent(2, 0, 2), (0, 0, 2): Agent(0, 0, 2), (3, 0, 2): Agent(3, 0, 2)}
***tick 5 rank 1 agent_cache {(2, 0, 0): Agent(2, 0, 0), (0, 0, 2): Agent(0, 0, 2), (3, 0, 2): Agent(3, 0, 2), (0, 0, 0): Agent(0, 0, 0), (1, 0, 0): Agent(1, 0, 0), (3, 0, 0): Agent(3, 0, 0), (2, 0, 2): Agent(2, 0, 2)}
***tick 5 rank 2 agent_cache {(0, 0, 0): Agent(0, 0, 0), (1, 0, 0): Agent(1, 0, 0), (1, 0, 1): Agent(1, 0, 1), (2, 0, 0): Agent(2, 0, 0), (2, 0, 1): Agent(2, 0, 1), (3, 0, 1): Agent(3, 0, 1), (3, 0, 0): Agent(3, 0, 0)}
***tick 5 rank 2 ghostsToRequest [((0, 0, 0), 0), ((1, 0, 0), 0), ((1, 0, 1), 1), ((2, 0, 0), 0), ((2, 0, 1), 1), ((3, 0, 1), 1), ((3, 0, 0), 0)]
***tick 5 rank 1 ghostsToRequest [((2, 0, 0), 0), ((0, 0, 2), 2), ((3, 0, 2), 2), ((0, 0, 0), 0), ((1, 0, 0), 0), ((3, 0, 0), 0), ((2, 0, 2), 2)]
***tick 5 rank 0 ghostsToRequest [((0, 0, 1), 1), (